In [2]:
import pandas as pd 
import numpy as np
import hvplot.pandas
import panel as pn
pn.extension('tabulator')

In [3]:
stats_df = pd.read_csv('../PBR_Analytics/Resources/BullRiders.csv')

In [5]:
stats_df.head()

,Rider,PBRrank,YearBorn,Height,Weight,YearsPro,Season,Events,BuckOuts,Rides,CupPoints,Place,RidePer,Wins,Top5,Top10,FinalPoints,Earnings
0,Silvano Alves,1,1987,67,150,5,2012,29,103,62,12201.75,1,0.60,2,8,18,1648.00,1464475.61
1,Marco Eguchi,2,1989,67,135,5,2012,29,84,39,7699.25,9,0.46,1,9,11,571.00,226255.50
2,Cody Nance,3,1987,68,162,7,2012,28,76,29,5068.25,16,0.38,1,4,7,322.00,131580.54
3,Shane Proctor,5,1985,68,165,8,2012,13,32,8,1387.50,35,0.25,0,0,3,0.00,52443.96
4,Fabian Vieria,7,1982,68,175,7,2012,23,70,38,7677.25,10,0.54,2,5,11,171.75,191399.08


In [7]:
#Previewing dtypes for potential errors
stats_df.dtypes

Rider           object
PBRrank          int64
YearBorn         int64
Height           int64
Weight           int64
YearsPro         int64
Season           int64
Events           int64
BuckOuts         int64
Rides            int64
CupPoints      float64
Place            int64
RidePer        float64
Wins             int64
Top5             int64
Top10            int64
FinalPoints    float64
Earnings       float64
dtype: object

In [ ]:
#Setting Up Interactive Panel

In [9]:
total_riders = len(stats_df.Rider.unique())
total_riders

In [36]:
#Rider score based on rides, wins diveded by Final points
rider_valuescore_df = pd.DataFrame().assign(Rider=stats_df['Rider'],
                                            Buckouts = stats_df['BuckOuts'],
                                            Rides= stats_df['Rides'],
                                            Wins=stats_df['Wins'],
                                            FinalPoints = stats_df['FinalPoints'],
                                            Earnings = stats_df['Earnings']) 

value_score =(rider_valuescore_df['Rides'] - rider_valuescore_df['Wins'])/rider_valuescore_df['FinalPoints']
rider_valuescore_df['Value_Score'] = value_score

In [39]:
rider_valuescore_df.sort_values('Value_Score', ascending=True)

,Rider,Buckouts,Rides,Wins,FinalPoints,Earnings,Value_Score
26,Robson Palermo,57,30,2,4189.25,466599.60,0.006684
15,Emilio Resende,45,24,0,3414.25,271470.63,0.007029
22,Renato Nunes,91,38,1,3298.50,254709.57,0.011217
13,Luke Snyder,85,31,1,2217.00,208540.07,0.013532
7,Guilherme Marchi,87,50,2,2519.75,331421.67,0.019050
29,Austin Meier,91,49,2,1773.25,311728.81,0.026505
8,Mike Lee,64,33,1,1011.75,147952.34,0.031628
0,Silvano Alves,103,62,2,1648.00,1464475.61,0.036408
14,L.J. Jenkins,93,49,1,1212.75,239935.66,0.039579
12,Kody Lostroh,61,23,1,525.00,158470.75,0.041905


In [46]:
value_score1 =((rider_valuescore_df['Rides'] - rider_valuescore_df['Wins'])/rider_valuescore_df['Earnings'])*1000
rider_valuescore_df['Value_ScoreE'] = value_score1

In [47]:
rider_valuescore_df.sort_values('Value_ScoreE', ascending=True)

,Rider,Buckouts,Rides,Wins,FinalPoints,Earnings,Value_Score,Value_ScoreE
0,Silvano Alves,103,62,2,1648.00,1464475.61,0.036408,0.040970
20,Brant Atwood,4,1,0,0.00,22059.59,inf,0.045332
36,Cody Johnson,21,6,1,0.00,88327.32,inf,0.056608
26,Robson Palermo,57,30,2,4189.25,466599.60,0.006684,0.060009
30,Lachlan Richardson,37,7,1,0.00,90044.06,inf,0.066634
27,Zack Brown,9,2,0,0.00,29790.72,inf,0.067135
15,Emilio Resende,45,24,0,3414.25,271470.63,0.007029,0.088407
25,Reese Cates,10,3,0,0.00,27987.21,inf,0.107192
9,Chase Outlaw,64,21,1,0.00,170203.48,inf,0.117506
5,J.B. Mauney,82,40,3,287.00,313340.27,0.128920,0.118082


In [57]:
#Scatter Plot by Earnings 
rider_valuescore_df.hvplot(x= 'Value_ScoreE',
                           y= 'Earnings',
                           kind = 'scatter',
                           c = 'Rider',
                           title = 'Value Score by Earnings vs Earnings')

:Scatter   [Value_ScoreE]   (Earnings,Rider)

In [59]:
#ScatterPlot by Points
rider_valuescore_df.hvplot(x= 'Value_Score',
                           y= 'FinalPoints',
                           kind = 'scatter',
                           c = 'Rider',
                           title = 'Value Score vs. Final Points')

:Scatter   [Value_Score]   (FinalPoints,Rider)

Top and Bottom 5 by Earnings 

In [69]:
#Top 5
rider_valuescore_df.sort_values('Earnings', ascending=False).head(5)

,Rider,Buckouts,Rides,Wins,FinalPoints,Earnings,Value_Score,Value_ScoreE
0,Silvano Alves,103,62,2,1648.00,1464475.61,0.036408,0.040970
26,Robson Palermo,57,30,2,4189.25,466599.60,0.006684,0.060009
7,Guilherme Marchi,87,50,2,2519.75,331421.67,0.019050,0.144831
5,J.B. Mauney,82,40,3,287.00,313340.27,0.128920,0.118082
29,Austin Meier,91,49,2,1773.25,311728.81,0.026505,0.150772


Top and Bottom 5 by Total Points

In [68]:
#Top 5
rider_valuescore_df.sort_values('FinalPoints', ascending=False).head(5)

,Rider,Buckouts,Rides,Wins,FinalPoints,Earnings,Value_Score,Value_ScoreE
26,Robson Palermo,57,30,2,4189.25,466599.60,0.006684,0.060009
15,Emilio Resende,45,24,0,3414.25,271470.63,0.007029,0.088407
22,Renato Nunes,91,38,1,3298.50,254709.57,0.011217,0.145263
7,Guilherme Marchi,87,50,2,2519.75,331421.67,0.019050,0.144831
13,Luke Snyder,85,31,1,2217.00,208540.07,0.013532,0.143857


Top 10 vs RidePer

In [71]:
stats_df.head(3)

,Rider,PBRrank,YearBorn,Height,Weight,YearsPro,Season,Events,BuckOuts,Rides,CupPoints,Place,RidePer,Wins,Top5,Top10,FinalPoints,Earnings
0,Silvano Alves,1,1987,67,150,5,2012,29,103,62,12201.75,1,0.60,2,8,18,1648.0,1464475.61
1,Marco Eguchi,2,1989,67,135,5,2012,29,84,39,7699.25,9,0.46,1,9,11,571.0,226255.50
2,Cody Nance,3,1987,68,162,7,2012,28,76,29,5068.25,16,0.38,1,4,7,322.0,131580.54


In [80]:
stats_df.hvplot.hist('RidePer', bins=30)

:Histogram   [RidePer]   (RidePer_count)